# GPU acceleration

## Overview

* **Why to bother with GPU computing in 2024**
  * HPC and Supercomputing is GPU-accelerated
  * When Julia overcomes the two-language barrier

* **GPU computing Fast-Forward**
  * Array vs Kernel programming
  * Performance considerations

* **Going multi-GPUs**
  * MPI + GPUs

### Why to still bother with GPU computing in 2024
- It's around for more than a decade
- It shows massive performance gain compared to serial CPU computing
- First exascale supercomputer, Frontier, is full of GPUs

![Frontier](imgs/frontier.png)

### Performance that matters

![cpu_gpu_evo](imgs/cpu_gpu_evo.png)

Taking a look at a recent GPU and CPU:
- Nvidia Tesla A100 GPU
- AMD EPYC "Rome" 7282 (16 cores) CPU

| Device         | TFLOP/s (FP64) | Memory BW TB/s | Imbalance (FP64)     |
| :------------: | :------------: | :------------: | :------------------: |
| Tesla A100     | 9.7            | 1.55           | 9.7 / 1.55  × 8 = 50 |
| AMD EPYC 7282  | 0.7            | 0.085          | 0.7 / 0.085 × 8 = 66 |

**Meaning:** we can do about 50 floating point operations per number accessed from main memory.
Floating point operations are "for free" when we work in memory-bounded regimes.

👉 Requires re-thinking the numerical implementation and solution strategies

Unfortunately, the cost of evaluating a first derivative $∂A / ∂x$ in, e.g., diffusive flux calculations using finite-differences:

`q[ix] = -D * (A[ix+1] - A[ix]) / dx`

consists of:
- 1 read (`A`) + 1 write (`q`) => $2 × 8$ = **16 Bytes transferred**
- 1 addition + 1 multiplication + 1 division => **3 floating point operations**

👉 assuming `D`, `dx` are scalars, `q` and `A` are arrays of `Float64` (read from main memory)

### Performance that matters - an example
Not yet convinced? Let's have a look at an example.

Let's assess how close from memory copy (1400 GB/s) we can get solving a 2D diffusion problem on an Nvidia Tesla A100 GPU.

$$ \frac{\partial C}{\partial t} = \frac{\partial^2 C}{\partial x^2} + \frac{\partial^2 C}{\partial y^2} $$

👉 Let's test the performance using a simple script.

### Measuring GPU performance

Load modules:

In [ ]:
using CUDA
using BenchmarkTools
using Printf

Memory copy function to measure the "peak" memory throughput:

In [ ]:
function mycopy!(A, B)
    ix = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    iy = (blockIdx().y - 1) * blockDim().y + threadIdx().y
    if ix <= size(A, 1) && iy <= size(A, 2)
        @inbounds A[ix, iy] = B[ix, iy]
    end
    return
end

Laplacian kernel using the finite difference method (FDM):

In [ ]:
function laplacian!(A, B, dt, _dx2, _dy2)
    ix = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    iy = (blockIdx().y - 1) * blockDim().y + threadIdx().y
    if ix <= size(A, 1) - 2 && iy <= size(A, 2) - 2
        @inbounds A[ix+1, iy+1] = B[ix+1, iy+1] + dt * 
            ((B[ix+2, iy+1] - 2 * B[ix+1, iy+1] + B[ix, iy+1]) * _dx2 +
             (B[ix+1, iy+2] - 2 * B[ix+1, iy+1] + B[ix+1, iy]) * _dy2)
    end
    return
end

Let's test the performance!

In [ ]:
# if the array size is too small, the GPU will not be fully utilized
nx = ny = 512 * 32
A = CUDA.rand(Float64, nx, ny)
B = CUDA.rand(Float64, nx, ny)

_dx2 = _dy2 = dt = rand()

# launch configuration
nthreads = (16, 16)
nblocks  = cld.((nx, ny), nthreads)

# measure the execution times
time_copy = @belapsed CUDA.@sync @cuda threads=nthreads blocks=nblocks mycopy!(A, B)
time_lapl = @belapsed CUDA.@sync @cuda threads=nthreads blocks=nblocks laplacian!(A, B, dt, _dx2, _dy2)

# effective memory throughput (1 read + 1 write per element)
Teff_copy = 2 * nx * ny * sizeof(Float64) / time_copy / 1e9
Teff_lapl = 2 * nx * ny * sizeof(Float64) / time_lapl / 1e9

# compute theoretical peak memory bandwidth
dev  = CUDA.device()

bus_width       = CUDA.attribute(dev, CUDA.CU_DEVICE_ATTRIBUTE_GLOBAL_MEMORY_BUS_WIDTH) |> Float64 # in bits
clock_rate      = CUDA.attribute(dev, CUDA.CU_DEVICE_ATTRIBUTE_MEMORY_CLOCK_RATE)       |> Float64 # in kHz
rate_multiplier = 2 # 2 for HBM2/DDR, 4 for HBM3/GDDR5, 8 for GDDR6

Teff_peak = bus_width * clock_rate * rate_multiplier / 1e6 / 8

# report results
@printf("Effective memory throughput (copy)      : %.2f GB/s\n", Teff_copy)
@printf("Effective memory throughput (laplacian) : %.2f GB/s\n", Teff_lapl)
@printf("Theoretical peak memory throughput      : %.2f GB/s\n", Teff_peak)

@printf("\nWow 🚀! Laplacian runs at:\n")
@printf("   %.2f%% of copy speed\n"           , 100 * Teff_lapl / Teff_copy)
@printf("   %.2f%% of peak memory bandwidth\n", 100 * Teff_lapl / Teff_peak)
@printf("on a %s device\n", CUDA.name(dev))